In [1]:
import cv2
import os
import numpy as np
import pandas as pd

In [23]:
def classes_csv_to_encoded_df(files: list, delimiter=";") -> pd.DataFrame:
    df_classes_all = pd.DataFrame()
    for file_path in files:
        df_classes = pd.read_csv(file_path, delimiter=delimiter)
        df_classes_all = pd.concat([df_classes_all,df_classes])
    encoded = pd.get_dummies(df_classes_all["Species"])
    df_classes_all.drop("Species", axis=1, inplace=True)
    df_classes_all.drop("Filename", axis=1, inplace=True)
    df_classes_all = df_classes_all.join(encoded)

    return df_classes_all

df_classes = classes_csv_to_encoded_df(["../Auswertung_20220524.csv","../LMU_20180326_class.csv"])

def id_to_class(id: int, df_classes: pd.DataFrame) -> pd.Series:
    for row in df_classes.iterrows():
        if id == row[1]["ID"]:
            return row[1].drop("ID")

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        if os.path.isfile(img_path):
            img = cv2.imread(img_path)
            if img is not None:
                x, y, width, height = 54, 35, 335, 217
                img = img[y:y+height, x:x+width]
                images.append([id_to_class(int(filename[:-4]),df_classes),img])
    return images

image_list = load_images_from_folder("../Bat_Orientation_Calls/")
image_array = np.array(image_list,dtype="object")

# flatten array
#image_array_flat = []
#for el in image_array:
#    image_array_flat.append([el[0],el[1].reshape(-1,3).reshape(-1)]) # second rehsape to flatten pixel
#image_array_flat = np.array(image_array_flat,dtype="object")

# Optionally, save the NumPy array to a file
#np.save('image_array.npy', image_array_flat)

#cv2.imshow("img",image_array[0][1])
#cv2.waitKey(0)

# convert to dataframe
images_df = pd.DataFrame(image_array,columns=['Image ID','Data'])
images_df.to_pickle('images.pkl')
images_flattend_df = pd.DataFrame(image_array,columns=['Image ID','Data'])
images_flattend_df.to_pickle('images_flatten.pkl')

#print(images_flattend_df)
print(images_flattend_df.iloc[0])
print(images_flattend_df[0][0])



KeyError: 0